In [10]:
import pandas as pd
import numpy as np

In [11]:
data = pd.read_csv('new_data/complete-data.csv')

data.sort_values('year')

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,...,weight,award_count,career_year,playoff_progression_rolling,playoff_rolling,pp36_rolling,eff_rolling,award_count_rolling,defensive_prowess_rolling,defensive_discipline_rolling
269,182,3,0,557,17.0,3.88,4.82,0.00,0.0,0.0,...,188,0.0,3,1.5,1.0,17.82,-272.590,0.0,97.20,11.91
94,71,3,0,566,32.0,1.62,2.97,0.00,0.0,0.0,...,218,0.0,3,0.0,0.0,16.20,-77.055,0.0,23.90,5.74
502,315,3,0,556,13.0,1.62,3.23,0.08,0.0,0.0,...,188,0.0,3,0.5,0.5,14.04,-253.835,0.0,74.20,10.09
92,68,3,0,555,32.0,1.44,4.38,0.00,22.0,5.0,...,185,0.0,3,0.5,0.5,10.26,-212.575,0.0,79.70,8.90
151,94,3,0,565,12.0,0.08,0.33,0.00,0.0,0.0,...,148,0.0,3,1.5,1.0,6.48,-36.590,0.0,8.15,2.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,372,11,0,564,0.0,0.00,0.00,0.00,0.0,0.0,...,162,0.0,4,0.5,0.5,14.94,-96.995,0.0,26.70,4.59
626,374,11,0,558,0.0,0.00,0.00,0.00,0.0,0.0,...,181,0.0,7,3.0,1.0,9.18,-100.480,0.0,33.20,7.25
629,375,11,0,558,0.0,0.00,0.00,0.00,0.0,0.0,...,160,0.0,5,2.0,0.5,23.04,-388.950,0.0,44.05,10.91
828,473,11,0,569,0.0,0.00,0.00,0.00,0.0,0.0,...,150,0.0,8,1.0,0.5,11.52,-103.950,0.0,13.00,3.16


In [12]:
with open('stats.txt', 'w') as file:
    for col in data.columns:
        if type(data[col][0]) == str: 
            continue
        file.write("-"*50 + "\n")
        file.write(f"Statistics of {col}\n")
        file.write("-"*50 + "\n")
        # data[col].plot.box();  # You can enable this if you want to create a plot

        q3, q1 = np.percentile(data[col], [75 ,25])

        file.write(f"Min: {data[col].min()}\n")
        file.write(f"Max: {data[col].max()}\n")
        file.write(f"Mean: {data[col].mean()}\n")
        file.write(f"Median: {data[col].median()}\n")
        file.write(f"Standard deviation: {data[col].std()}\n")
        file.write(f"Skewness: {data[col].skew()}\n")
        file.write(f"Variance: {data[col].var()}\n")
        file.write(f"Kurtosis: {data[col].kurtosis()}\n")
        file.write(f"Interquartile Range: {q3 - q1} | Q1: {q1} | Q3: {q3}\n")

In [13]:
outliers_std = {}
all_outliers = []  # Array to store all outlier rows

#columns to skip
not_skip_cols = ['height', 'weight']

for col in data.columns:
    if type(data[col][0]) == str or col not in not_skip_cols: 
        continue
    data_mean, data_std = data[col].mean(), data[col].std()
    # identify outliers
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    outliers = pd.concat([data.loc[data[col] < lower], data.loc[data[col] > upper]])
    outliers_std[col] = outliers
    all_outliers.extend(outliers.values.tolist())  # Add outliers to the array

for key in outliers_std.keys():
    if len(outliers_std[key]) > 0:
        print("-"*50)
        print(f"Outliers of {key} - mean: {data[key].mean()} - std: {data[key].std()}")
        print("-"*50)
        print(outliers_std[key][key])

# Printing all outlier rows
print("All outlier rows:")
for outlier in all_outliers:
    print(outlier)

--------------------------------------------------
Outliers of height - mean: 71.63685932388222 - std: 6.425426612759096
--------------------------------------------------
204    9.0
205    9.0
206    9.0
207    9.0
208    9.0
209    9.0
210    9.0
Name: height, dtype: float64
--------------------------------------------------
Outliers of weight - mean: 167.99345692475464 - std: 22.764168790496495
--------------------------------------------------
697      0
221    240
300    240
301    240
302    240
384    240
385    240
386    240
387    240
Name: weight, dtype: int64
All outlier rows:
[130.0, 3.0, 0.0, 553.0, 30.0, 1.73, 7.0, 0.07, 34.2, 12.0, 0.6, 8.2, 8.8, 2.4, 0.2, 3.4, 3.2, 3.4, 0.0, 0.84, 0.44, 0.25, 0.9, 0.87, 0.4, 0.6, 1.0, 0.88, 1.04, 13.13, 8.73, 2.37, 0.83, 3.57, -268.37, 16.2, 114.0, 13.0, 29.2, 3.0, 578.0, 1.0, 697.0, 0.0, 9.0, 223.0, 0.0, 3.0, 0.5, 0.5, 13.32, -232.19, 0.0, 95.6, 12.31]
[130.0, 4.0, 0.0, 560.0, 34.0, 1.32, 6.06, 0.03, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [14]:
outliers_ids_std = set([val for key in outliers_std.keys() for val in outliers_std[key]['playerID']])
outlier_ratio = len(outliers_ids_std) / len(data)
outlier_ratio

0.0054525627044711015

In [15]:
outliers_z = {}
for col in data.columns:
    if type(data[col][0]) == str or col not in not_skip_cols: 
        continue
    z_val = (data[col] - data[col].mean()) / data[col].std()
    outliers_z[col] = data.loc[(z_val < -4) | (z_val > 4)]

for key in outliers_z.keys():
    if (len(outliers_z[key]) > 0):
        print("-"*50)
        print(f"Outliers of {key} - mean: {data[key].mean()} - std: {data[key].std()}")
        print("-"*50)
        print(outliers_z[key][key])

--------------------------------------------------
Outliers of height - mean: 71.63685932388222 - std: 6.425426612759096
--------------------------------------------------
204    9.0
205    9.0
206    9.0
207    9.0
208    9.0
209    9.0
210    9.0
Name: height, dtype: float64
--------------------------------------------------
Outliers of weight - mean: 167.99345692475464 - std: 22.764168790496495
--------------------------------------------------
697    0
Name: weight, dtype: int64


In [16]:
outliers_ids_z = set([val for key in outliers_z.keys() for val in outliers_z[key]['playerID']])
outlier_ratio = len(outliers_ids_z) / len(data)
outlier_ratio

0.0021810250817884407

In [17]:
outliers_ids = outliers_ids_std.union(outliers_ids_z)
print(f"Total outliers: {len(outliers_ids)}")
print(f"Total outliers ratio: {len(outliers_ids) / len(data)}")
print(outliers_ids)
#create a new df with outliers removed
manual_outlier_df = data[~data['playerID'].isin(outliers_ids)]

manual_outlier_df.duplicated(subset=['playerID', 'year', 'stint']).sum()

Total outliers: 5
Total outliers ratio: 0.0054525627044711015
{241, 130, 196, 408, 137}


3

In [18]:

manual_outlier_df.to_csv('new_data/complete-data.csv', index=False)
manual_outlier_df.sort_values('year')
# manual_outlier_df.to_csv('output/loan_dev_transformed_without_outliers.csv', index=False)

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,...,weight,award_count,career_year,playoff_progression_rolling,playoff_rolling,pp36_rolling,eff_rolling,award_count_rolling,defensive_prowess_rolling,defensive_discipline_rolling
655,392,3,0,567,5.0,0.20,0.60,0.00,0.0,0.0,...,168,0.0,3,0.5,0.5,14.22,-149.875,0.0,48.60,4.44
654,391,3,0,559,10.0,0.80,0.50,0.00,0.0,0.0,...,164,0.0,3,0.0,0.0,12.78,-151.800,0.0,20.45,8.04
355,225,3,0,554,5.0,1.40,1.20,0.00,0.0,0.0,...,179,0.0,3,2.5,1.0,8.28,-67.700,0.0,24.15,5.53
356,226,3,0,566,32.0,0.62,1.22,0.00,0.0,0.0,...,118,0.0,3,0.0,0.0,12.60,-120.680,0.0,21.15,5.50
366,236,3,0,553,28.0,0.29,0.39,0.04,8.0,3.4,...,150,0.0,3,0.5,0.5,9.36,-89.410,0.0,12.40,4.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,243,11,0,552,0.0,0.00,0.00,0.00,0.0,0.0,...,196,0.0,9,0.5,0.5,19.98,-267.790,0.0,53.75,10.39
411,256,11,0,569,0.0,0.00,0.00,0.00,0.0,0.0,...,180,0.0,6,1.0,0.5,12.42,-228.340,0.0,54.55,8.59
414,258,11,0,741,0.0,0.00,0.00,0.00,0.0,0.0,...,175,0.0,5,0.5,0.5,10.44,-131.025,0.0,20.25,6.59
754,438,11,0,558,0.0,0.00,0.00,0.00,0.0,0.0,...,183,0.0,4,1.0,0.5,12.24,-82.975,0.0,13.65,3.15
